In [1]:
import numpy as np
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1 ,freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

train_X = series[:7500, :n_steps]
valid_X = series[7500:, :n_steps]

y = np.empty((10000, n_steps, 1))
y[:, :, 0] = series[:, 1:, 0]

train_y = y[:7500]
valid_y = y[7500:]

In [4]:
valid_pred_naive_y = valid_X
np.mean(keras.losses.mean_squared_error(valid_y, valid_pred_naive_y))

0.020491822

In [5]:
early_stop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

def last_time_step_mse(y_true, y_pred):
    return keras.metrics.mean_squared_error(y_true[:, -1], y_pred[:, -1])

In [6]:
model_linear = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model_linear.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_linear.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 0s 1ms/step - loss: 0.1610 - last_time_step_mse: 0.1779 - val_loss: 0.1472 - val_last_time_step_mse: 0.1576
Epoch 2/50
235/235 [==============================] - 0s 675us/step - loss: 0.1464 - last_time_step_mse: 0.1521 - val_loss: 0.1463 - val_last_time_step_mse: 0.1558
Epoch 3/50
235/235 [==============================] - 0s 747us/step - loss: 0.1461 - last_time_step_mse: 0.1510 - val_loss: 0.1461 - val_last_time_step_mse: 0.1563
Epoch 4/50
235/235 [==============================] - 0s 702us/step - loss: 0.1459 - last_time_step_mse: 0.1503 - val_loss: 0.1459 - val_last_time_step_mse: 0.1540
Epoch 5/50
235/235 [==============================] - 0s 712us/step - loss: 0.1458 - last_time_step_mse: 0.1496 - val_loss: 0.1457 - val_last_time_step_mse: 0.1542
Epoch 6/50
235/235 [==============================] - 0s 742us/step - loss: 0.1456 - last_time_step_mse: 0.1491 - val_loss: 0.1456 - val_last_time_step_mse: 0.1536
Epoch 7/50
235/235

In [7]:
model_baseline = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(1)
])

model_baseline.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_baseline.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 1s 1ms/step - loss: 0.1546 - last_time_step_mse: 0.1576 - val_loss: 0.1468 - val_last_time_step_mse: 0.1489
Epoch 2/50
235/235 [==============================] - 0s 793us/step - loss: 0.1459 - last_time_step_mse: 0.1481 - val_loss: 0.1456 - val_last_time_step_mse: 0.1503
Epoch 3/50
235/235 [==============================] - 0s 808us/step - loss: 0.1453 - last_time_step_mse: 0.1481 - val_loss: 0.1452 - val_last_time_step_mse: 0.1500
Epoch 4/50
235/235 [==============================] - 0s 826us/step - loss: 0.1450 - last_time_step_mse: 0.1482 - val_loss: 0.1449 - val_last_time_step_mse: 0.1527
Epoch 5/50
235/235 [==============================] - 0s 789us/step - loss: 0.1448 - last_time_step_mse: 0.1483 - val_loss: 0.1448 - val_last_time_step_mse: 0.1528
Epoch 6/50
235/235 [==============================] - 0s 803us/step - loss: 0.1446 - last_time_step_mse: 0.1482 - val_loss: 0.1447 - val_last_time_step_mse: 0.1497
Epoch 7/50
235/235

In [8]:
model_simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(1, return_sequences=True, input_shape=[None, 1])
])

model_simpleRNN.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_simpleRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 2s 5ms/step - loss: 0.0177 - last_time_step_mse: 0.0152 - val_loss: 0.0140 - val_last_time_step_mse: 0.0116
Epoch 2/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0133 - last_time_step_mse: 0.0118 - val_loss: 0.0127 - val_last_time_step_mse: 0.0107
Epoch 3/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0126 - last_time_step_mse: 0.0114 - val_loss: 0.0123 - val_last_time_step_mse: 0.0105
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0122 - last_time_step_mse: 0.0113 - val_loss: 0.0121 - val_last_time_step_mse: 0.0105
Epoch 5/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0121 - last_time_step_mse: 0.0113 - val_loss: 0.0120 - val_last_time_step_mse: 0.0105
Epoch 6/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0120 - last_time_step_mse: 0.0113 - val_loss: 0.0120 - val_last_time_step_mse: 0.0105
Epoch 7/50
235/235 [========

In [9]:
model_deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(1))
])

model_deepRNN.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_deepRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 13ms/step - loss: 0.0181 - last_time_step_mse: 0.0122 - val_loss: 0.0061 - val_last_time_step_mse: 0.0039
Epoch 2/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0055 - last_time_step_mse: 0.0035 - val_loss: 0.0052 - val_last_time_step_mse: 0.0033
Epoch 3/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0051 - last_time_step_mse: 0.0033 - val_loss: 0.0050 - val_last_time_step_mse: 0.0032
Epoch 4/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0050 - last_time_step_mse: 0.0033 - val_loss: 0.0049 - val_last_time_step_mse: 0.0031
Epoch 5/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0049 - last_time_step_mse: 0.0032 - val_loss: 0.0048 - val_last_time_step_mse: 0.0031
Epoch 6/50
235/235 [==============================] - 3s 13ms/step - loss: 0.0048 - last_time_step_mse: 0.0032 - val_loss: 0.0047 - val_last_time_step_mse: 0.0030
Epoch 7/50
235/235 [==

In [10]:
model_singleLSTM = keras.models.Sequential([
    keras.layers.LSTM(1, return_sequences=True, input_shape=[None,1]),
])

model_singleLSTM.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_singleLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 3s 10ms/step - loss: 0.0887 - last_time_step_mse: 0.0893 - val_loss: 0.0771 - val_last_time_step_mse: 0.0795
Epoch 2/50
235/235 [==============================] - 2s 8ms/step - loss: 0.0668 - last_time_step_mse: 0.0675 - val_loss: 0.0577 - val_last_time_step_mse: 0.0597
Epoch 3/50
235/235 [==============================] - 2s 8ms/step - loss: 0.0502 - last_time_step_mse: 0.0514 - val_loss: 0.0438 - val_last_time_step_mse: 0.0452
Epoch 4/50
235/235 [==============================] - 2s 9ms/step - loss: 0.0386 - last_time_step_mse: 0.0397 - val_loss: 0.0342 - val_last_time_step_mse: 0.0351
Epoch 5/50
235/235 [==============================] - 2s 9ms/step - loss: 0.0306 - last_time_step_mse: 0.0316 - val_loss: 0.0276 - val_last_time_step_mse: 0.0281
Epoch 6/50
235/235 [==============================] - 2s 9ms/step - loss: 0.0251 - last_time_step_mse: 0.0258 - val_loss: 0.0229 - val_last_time_step_mse: 0.0231
Epoch 7/50
235/235 [=======

In [11]:
model_deepLSTM = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(1))
])

model_deepLSTM.compile(loss='mean_squared_error', optimizer='Adam', metrics=[last_time_step_mse])
model_deepLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 7s 23ms/step - loss: 0.0455 - last_time_step_mse: 0.0356 - val_loss: 0.0099 - val_last_time_step_mse: 0.0037
Epoch 2/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0078 - last_time_step_mse: 0.0037 - val_loss: 0.0066 - val_last_time_step_mse: 0.0035
Epoch 3/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0060 - last_time_step_mse: 0.0035 - val_loss: 0.0055 - val_last_time_step_mse: 0.0034
Epoch 4/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0052 - last_time_step_mse: 0.0033 - val_loss: 0.0048 - val_last_time_step_mse: 0.0031
Epoch 5/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0046 - last_time_step_mse: 0.0031 - val_loss: 0.0046 - val_last_time_step_mse: 0.0031
Epoch 6/50
235/235 [==============================] - 5s 20ms/step - loss: 0.0043 - last_time_step_mse: 0.0030 - val_loss: 0.0041 - val_last_time_step_mse: 0.0028
Epoch 7/50
235/235 [==